# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.



In [2]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter

In [18]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Importing Our List of Stocks


In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [5]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe.

In [9]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2999148897360,
 'week52high': 194.48,
 'week52low': 123.64,
 'week52highSplitAdjustOnly': 194.48,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.3079866896508676,
 'sharesOutstanding': 15728702000,
 'float': 0,
 'avg10Volume': 50318332,
 'avg30Volume': 54650515,
 'day200MovingAvg': 159.99,
 'day50MovingAvg': 183.03,
 'employees': 164000,
 'ttmEPS': 5.88,
 'ttmDividendRate': 0.9264396314120724,
 'dividendYield': 0.00485860935290577,
 'nextDividendDate': '',
 'exDividendDate': '2023-05-12',
 'nextEarningsDate': '2023-08-03',
 'peRatio': 31.7972550901708,
 'beta': 1.2641963363742728,
 'maxChangePercent': 73.23788203231459,
 'year5ChangePercent': 3.257105222031212,
 'year2ChangePercent': 0.3379343831173842,
 'year1ChangePercent': 0.3141535467259191,
 'ytdChangePercent': 0.4738687431303712,
 'month6ChangePercent': 0.4773939663007274,
 'month3ChangePercent': 0.16122855886590814,
 'month1ChangePercent': 0.07232032392306831,
 'day30ChangePerc

In [17]:
data['ytdChangePercent']

0.4738687431303712

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

## Executing A Batch API Call & Building Our DataFrame


In [21]:

def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [55]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    
    data = requests.get(batch_api_call_url).json()
#     print(data['AAPL']['price'])
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        final_dataframe = final_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ], index = my_columns), ignore_index = True)

In [56]:
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,117,-0.038503,N/A
1,AAL,18.35,0.313529,N/A
2,AAP,69.38,-0.608406,N/A
3,AAPL,190.68,0.307987,N/A
4,ABBV,135.5,-0.079126,N/A
...,...,...,...,...
496,YUM,133.45,0.154055,N/A
497,ZBH,140.86,0.347526,N/A
498,ZBRA,295.22,-0.014356,N/A
499,ZION,28.98,-0.398478,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [61]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True , inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,N/A
1,388,RCL,102.01,1.961963,N/A
2,193,FTI,17.14,1.694969,N/A
3,343,NVDA,425.03,1.686106,N/A
4,330,NFLX,438.1,1.343094,N/A
5,197,GE,108.27,1.181869,N/A
6,84,CCL,19.04,1.113208,N/A
7,326,NCLH,21.89,0.898526,N/A
8,77,CAH,93.97,0.821894,N/A
9,489,WYNN,101.32,0.802706,N/A


## Calculating the Number of Shares to Buy



In [62]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:25000
25000


In [63]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,6
1,388,RCL,102.01,1.961963,4
2,193,FTI,17.14,1.694969,29
3,343,NVDA,425.03,1.686106,1
4,330,NFLX,438.1,1.343094,1
5,197,GE,108.27,1.181869,4
6,84,CCL,19.04,1.113208,26
7,326,NCLH,21.89,0.898526,22
8,77,CAH,93.97,0.821894,5
9,489,WYNN,101.32,0.802706,4


## Building a Better (and More Realistic) Momentum Strategy


* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns


In [154]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)
    
    
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   np.nan,
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   np.nan,
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   np.nan,
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   np.nan,
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   np.nan,
                                                   np.nan
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
    

    

In [155]:
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117,NaN,-0.038503,NaN,-0.204886,NaN,-0.150739,NaN,0.022089,NaN,NaN
1,AAL,18.35,NaN,0.313529,NaN,0.294076,NaN,0.309779,NaN,0.199346,NaN,NaN
2,AAP,69.38,NaN,-0.608406,NaN,-0.543901,NaN,-0.417943,NaN,0.071671,NaN,NaN
3,AAPL,190.68,NaN,0.307987,NaN,0.477394,NaN,0.161229,NaN,0.055989,NaN,NaN
4,ABBV,135.5,NaN,-0.079126,NaN,-0.171141,NaN,-0.153534,NaN,-0.015548,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,133.45,NaN,0.154055,NaN,0.029033,NaN,0.014933,NaN,0.001877,NaN,NaN
497,ZBH,140.86,NaN,0.347526,NaN,0.12461,NaN,0.092314,NaN,0.042902,NaN,NaN
498,ZBRA,295.22,NaN,-0.014356,NaN,0.079731,NaN,-0.016785,NaN,0.061828,NaN,NaN
499,ZION,28.98,NaN,-0.398478,NaN,-0.410032,NaN,0.020983,NaN,-0.094516,NaN,NaN


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`


In [156]:
time_periods= [
            'One-Year',
            'Six-Month',
            'Three-Month',
            'One-Month'
            ]
hqm_dataframe.replace([None], 0, inplace = True)
for row in hqm_dataframe.index:
    for time_period in time_periods:
        
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        
        hqm_dataframe.loc[row, percentile_col] = stats.percentileofscore(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.00,0,-0.038503,0.267465,-0.204886,0.059880,-0.150739,0.049900,0.022089,0.518962,0
1,AAL,18.35,0,0.313529,0.806387,0.294076,0.912176,0.309779,0.978044,0.199346,0.988024,0
2,AAP,69.38,0,-0.608406,0.003992,-0.543901,0.003992,-0.417943,0.003992,0.071671,0.846307,0
3,AAPL,190.68,0,0.307987,0.802395,0.477394,0.968064,0.161229,0.840319,0.055989,0.762475,0
4,ABBV,135.50,0,-0.079126,0.203593,-0.171141,0.109780,-0.153534,0.045908,-0.015548,0.203593,0
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,133.45,0,0.154055,0.614770,0.029033,0.546906,0.014933,0.459082,0.001877,0.365269,0
497,ZBH,140.86,0,0.347526,0.826347,0.124610,0.750499,0.092314,0.676647,0.042902,0.674651,0
498,ZBRA,295.22,0,-0.014356,0.299401,0.079731,0.660679,-0.016785,0.325349,0.061828,0.790419,0
499,ZION,28.98,0,-0.398478,0.019960,-0.410032,0.011976,0.020983,0.473054,-0.094516,0.021956,0


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.


In [157]:
from statistics import mean


for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe
        

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,117.00,0,-0.038503,0.267465,-0.204886,0.059880,-0.150739,0.049900,0.022089,0.518962,0.224052
1,AAL,18.35,0,0.313529,0.806387,0.294076,0.912176,0.309779,0.978044,0.199346,0.988024,0.921158
2,AAP,69.38,0,-0.608406,0.003992,-0.543901,0.003992,-0.417943,0.003992,0.071671,0.846307,0.214571
3,AAPL,190.68,0,0.307987,0.802395,0.477394,0.968064,0.161229,0.840319,0.055989,0.762475,0.843313
4,ABBV,135.50,0,-0.079126,0.203593,-0.171141,0.109780,-0.153534,0.045908,-0.015548,0.203593,0.140719
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,133.45,0,0.154055,0.614770,0.029033,0.546906,0.014933,0.459082,0.001877,0.365269,0.496507
497,ZBH,140.86,0,0.347526,0.826347,0.124610,0.750499,0.092314,0.676647,0.042902,0.674651,0.732036
498,ZBRA,295.22,0,-0.014356,0.299401,0.079731,0.660679,-0.016785,0.325349,0.061828,0.790419,0.518962
499,ZION,28.98,0,-0.398478,0.019960,-0.410032,0.011976,0.020983,0.473054,-0.094516,0.021956,0.131737


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [158]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(inplace = True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CCL,19.04,0,1.113208,0.988024,1.069565,0.998004,0.956835,1.000000,0.493333,1.000000,0.996507
1,NCLH,21.89,0,0.898526,0.986028,0.678681,0.990020,0.673547,0.998004,0.302201,0.998004,0.993014
2,RCL,102.01,0,1.961963,0.998004,0.855065,0.996008,0.664654,0.996008,0.130430,0.970060,0.990020
3,NVDA,425.03,0,1.686106,0.994012,1.861500,1.000000,0.572357,0.994012,0.103687,0.936128,0.981038
4,FTI,17.14,0,1.694969,0.996008,0.377814,0.948104,0.276247,0.960080,0.153432,0.978044,0.970559
5,DAL,47.88,0,0.611036,0.960080,0.328893,0.922156,0.421193,0.992016,0.224239,0.990020,0.966068
6,LB,79.92,0,2.281287,1.000000,0.815831,0.994012,0.215421,0.906188,0.079271,0.874251,0.943613
7,UAL,55.39,0,0.518783,0.928144,0.311316,0.916168,0.298406,0.972056,0.110688,0.956088,0.943114
8,GWW,776.18,0,0.657164,0.968064,0.411863,0.958084,0.213024,0.902196,0.100996,0.924152,0.938124
9,NFLX,438.10,0,1.343094,0.992016,0.388370,0.950100,0.291074,0.968064,0.070181,0.840319,0.937625


## Calculating the Number of Shares to Buy

In [127]:
portfolio_input()

Enter the value of your portfolio:1000000


In [164]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/hqm_dataframe.loc[i, "Price"]) 
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,CCL,19.04,1029,1.113208,0.988024,1.069565,0.998004,0.956835,1.000000,0.493333,1.000000,0.996507
1,NCLH,21.89,895,0.898526,0.986028,0.678681,0.990020,0.673547,0.998004,0.302201,0.998004,0.993014
2,RCL,102.01,192,1.961963,0.998004,0.855065,0.996008,0.664654,0.996008,0.130430,0.970060,0.990020
3,NVDA,425.03,46,1.686106,0.994012,1.861500,1.000000,0.572357,0.994012,0.103687,0.936128,0.981038
4,FTI,17.14,1143,1.694969,0.996008,0.377814,0.948104,0.276247,0.960080,0.153432,0.978044,0.970559
5,DAL,47.88,409,0.611036,0.960080,0.328893,0.922156,0.421193,0.992016,0.224239,0.990020,0.966068
6,LB,79.92,245,2.281287,1.000000,0.815831,0.994012,0.215421,0.906188,0.079271,0.874251,0.943613
7,UAL,55.39,353,0.518783,0.928144,0.311316,0.916168,0.298406,0.972056,0.110688,0.956088,0.943114
8,GWW,776.18,25,0.657164,0.968064,0.411863,0.958084,0.213024,0.902196,0.100996,0.924152,0.938124
9,NFLX,438.10,44,1.343094,0.992016,0.388370,0.950100,0.291074,0.968064,0.070181,0.840319,0.937625


## Formatting Our Excel Output



In [169]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

In [170]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [171]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', percent_template]
                    }

for column in column_formats:
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [172]:
writer.save()